#Creating source and destination tables

In [0]:
-- Create the source table if it does not already exist
CREATE TABLE IF NOT EXISTS scd_cat.scd_schema.inc_source (
  id INT,         -- Unique identifier for each record
  name STRING,    -- Name field (string type)
  value INT       -- Value field (integer type)
);

-- Create the target table if it does not already exist
CREATE TABLE IF NOT EXISTS scd_cat.scd_schema.inc_target (
  id INT,         -- Unique identifier for each record
  name STRING,    -- Name field (string type)
  value INT       -- Value field (integer type)
);

##No data in source or destination table

In [0]:
select
  *
from
  scd_cat.scd_schema.inc_source;

select
  *
from
  scd_cat.scd_schema.inc_target;

##Adding sample data in source table

In [0]:
insert into scd_cat.scd_schema.inc_source
  values (1, 'a', 1), (2, 'b', 2), (3, 'c', 3);

select
  *
from
  scd_cat.scd_schema.inc_source;

#Using MERGE logic for incremental load

In [0]:
-- Merge data from inc_source into inc_target using the 'id' as the key
MERGE INTO
  scd_cat.scd_schema.inc_target AS target -- Target table to update or insert into
USING
  scd_cat.scd_schema.inc_source AS source -- Source table with new data
ON
  target.id = source.id -- Match records based on 'id'
WHEN MATCHED AND
  (
    source.name <> target.name
    or source.value <> target.value
  ) -- If a match is found and either 'name' or 'value' is different
  THEN UPDATE SET target.name = source.name, target.value = source.value -- Update the differing fields
WHEN NOT MATCHED THEN INSERT (id, name, value) VALUES (source.id, source.name, source.value); -- Insert new records not present in target

##Checking data after running merge

In [0]:
select * from scd_cat.scd_schema.inc_target

In [0]:
insert into scd_cat.scd_schema.inc_source
  values (4, 'd', 4), (5, 'e', 5);

update
  scd_cat.scd_schema.inc_source
set
  value = 100
where
  id = 1;